In [11]:
from keras.models import Sequential
from keras.layers import BatchNormalization, Dropout, Dense, Flatten, Reshape, LeakyReLU, Conv2D, Conv2DTranspose
from tensorflow.keras.optimizers import Adam
import numpy as np


In [ ]:
!mkdir gen_cifar_images

In [13]:
img_width= 32
img_height= 32
channels= 3
img_shape= (img_width, img_height, channels)
latent_dim= 100
adam= Adam(learning_rate= 0.0002)

In [3]:
from keras.datasets import cifar10

In [9]:
def build_generator():
    model = Sequential()
    model.add(Dense(256 * 4 * 4, input_dim= latent_dim))
    model.add(LeakyReLU(alpha= 0.2))
    model.add(Reshape((4, 4, 256)))
    model.add(Conv2DTranspose(128, (4,4) , strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha= 0.2))
    model.add(Conv2DTranspose(128, (4,4) , strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha= 0.2))
    model.add(Conv2DTranspose(128, (4,4) , strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha= 0.2))
    model.add(Conv2D(3, (3,3), activation= 'tanh', padding= 'same'))
    
    model.summary()
    return model

generator= build_generator()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 4096)              413696    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 4096)              0         
                                                                 
 reshape (Reshape)           (None, 4, 4, 256)         0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 8, 8, 128)        524416    
 nspose)                                                         
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 8, 8, 128)         0         
                                                                 
 conv2d_transpose_1 (Conv2DT  (None, 16, 16, 128)      262272    
 ranspose)                                            

In [15]:
def build_discriminator():
    model = Sequential()
    model.add(Conv2D(64, (3,3), padding='same', input_shape= img_shape))
    model.add(LeakyReLU(0.2))
    model.add(Conv2D(128, (3,3), padding='same'))
    model.add(LeakyReLU(0.2))
    model.add(Conv2D(128, (3,3), padding='same'))
    model.add(LeakyReLU(0.2))
    model.add(Conv2D(256, (3,3), padding='same'))
    model.add(LeakyReLU(0.2))

    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation= 'sigmoid'))    
    
    model. summary()
    return model

discriminator= build_discriminator()
discriminator.compile(optimizer= 'adam', loss= 'binary_crossentropy', metrics=['accuracy'])

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_13 (Conv2D)          (None, 32, 32, 64)        1792      
                                                                 
 leaky_re_lu_17 (LeakyReLU)  (None, 32, 32, 64)        0         
                                                                 
 conv2d_14 (Conv2D)          (None, 32, 32, 128)       73856     
                                                                 
 leaky_re_lu_18 (LeakyReLU)  (None, 32, 32, 128)       0         
                                                                 
 conv2d_15 (Conv2D)          (None, 32, 32, 128)       147584    
                                                                 
 leaky_re_lu_19 (LeakyReLU)  (None, 32, 32, 128)       0         
                                                                 
 conv2d_16 (Conv2D)          (None, 32, 32, 256)      

In [16]:

gan = Sequential()
gan.add(generator)
discriminator.trainable= False
gan.add(discriminator)

gan.compile(optimizer= 'adam', loss= 'binary_crossentropy', metrics=['accuracy'])

In [1]:
import matplotlib.pyplot as plt
import glob
import imageio
import PIL

save_name= 0.00000000

def save_image(epoch):
    r, c = 5, 5 
    noise= np.random.normal(0,1, (r*c, latent_dim))
    gen_imgs= generator.predict(noise)
    global save_name
    save_name+= 0.00000001
    print("%.8f" % save_name)
    gen_imgs= 0.5 * gen_imgs + 0.5
    fig, axs= plt.subplots(r,c)
    cnt= 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt,:,:,0],cmap= 'gray')
            axs[i,j].axis('off')
            cnt+=1
    fig.savefig("gen_cifar_images/%.8f.png" % save_name)
    plt.close()


In [4]:
(x,_),(_,_)= cifar10.load_data()
x.shape

(50000, 32, 32, 3)

In [22]:
def train(epochs, batch_size= 64, save_interval= 200):
    (x,_),(_,_)= cifar10.load_data()
    x= x/ 127.5 -1.
    valid= np.ones((batch_size, 1))
    fake= np.zeros((batch_size, 1))
    batch_per_epoch= int(x.shape[0] /batch_size)
    
    for epoch in range(epochs):
        for j in range(batch_per_epoch):
            idx= np.random.randint(0, x.shape[0], batch_size)
            imgs= x[idx]

            noise= np.random.normal(0,1, (batch_size, latent_dim))
            gen_imgs= generator.predict(noise)

            # train discriminator
            d_loss_real= discriminator.train_on_batch(imgs, valid)
            d_loss_fake= discriminator.train_on_batch(gen_imgs, fake)
            d_loss= np.add(d_loss_real,d_loss_fake) * 0.5

            noise= np.random.normal(0, 1, (batch_size, latent_dim))
            g_loss= gan.train_on_batch(noise, valid)

            print(f'{epoch} [D loss {d_loss} ] [G loss {g_loss}]')
        
        save_image(epoch)
        
train(30000)

2/2 [==============================] - 1s 405ms/step
0 [D loss [1.12672758 0.8671875 ] ] [G loss [7.699062824249268, 0.0]]
2/2 [==============================] - 1s 485ms/step
0 [D loss [0.20236823 0.9375    ] ] [G loss [4.526032447814941, 0.0]]
2/2 [==============================] - 2s 736ms/step
0 [D loss [0.04727246 1.        ] ] [G loss [2.0936551094055176, 0.0]]
2/2 [==============================] - 1s 388ms/step
0 [D loss [0.66562444 0.5       ] ] [G loss [1.4812076091766357, 0.0]]
2/2 [==============================] - 1s 339ms/step
0 [D loss [0.14687881 1.        ] ] [G loss [3.483320951461792, 0.0]]
2/2 [==============================] - 1s 374ms/step


KeyboardInterrupt: 

In [ ]:
anim_file = 'dcgan.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob('generated_cifar_images/*.png')
  filenames = sorted(filenames)
  for filename in filenames:
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)